# Work with lyrics 

## Make lyrics inverted index

In [4]:
# Get lyrics in BOW format:
# !cd data
# !wget http://millionsongdataset.com/sites/default/files/AdditionalFiles/mxm_dataset_train.txt.zip
# !wget http://millionsongdataset.com/sites/default/files/AdditionalFiles/mxm_dataset_test.txt.zip

In [5]:
import pickle
from lyrics_dictionary_initialisation import read_lyrics, make_inverted_index, save_lyrics_inverted_index

# LOAD DICTS
lyrics_test_path = 'data/mxm_dataset_test.txt'
lyrics_train_path = 'data/mxm_dataset_train.txt'

# Output
lyrics_all_words_path = 'data/lyrics_all_words.txt'
lyrics_inverted_index_word = 'data/lyrics_inverted_idx.pkl'

# Load the data
lyrics_dict_idx, all_words = read_lyrics(lyrics_train_path, lyrics_test_path)

# Make mappings from words to indices, where idx starts at 1
word_to_idx = {word: index+1 for index, word in enumerate(all_words)}
index_to_word = {index+1: word for index, word in enumerate(all_words)}

# Create a dictionary with words instead of indices
lyrics_dict_word = {track_id: {index_to_word[int(id)]: int(freq) for id, freq in word_dict.items()} for track_id, word_dict in lyrics_dict_idx.items()}

# Create inverted index
inverted_index = make_inverted_index(lyrics_dict_word, all_words)

# Save the inverted index
save_lyrics_inverted_index(inverted_index)

# Load the data
with open('data/lyrics_inverted_word.pkl', 'rb') as file:
    inverted_index = pickle.load(file)

lyrics_inverted_index saved as pickle at data/lyrics_inverted_word.pkl


In [6]:
# SHOW DATA
print('Number of words:', len(all_words))
print('Number of tracks:', len(lyrics_dict_idx))
print('First 5 words:', all_words[:5])
print('One example of the index dictionary')
for track_id, word_dict in lyrics_dict_idx.items():
    print(f"\t{track_id}: {word_dict}")
    break
print('One example of the word dictionary')
for track_id, word_dict in lyrics_dict_word.items():
    print(f"\t{track_id}: {word_dict}")
    break

print('One example of the inversed index')
word_example = 'chocol'
print(f"{word_example} ({len(inverted_index[word_example])} tracks):", {k: v for k, v in list(inverted_index[word_example].items())[:5]})

Number of words: 5000
Number of tracks: 237662
First 5 words: ['i', 'the', 'you', 'to', 'and']
One example of the index dictionary
	TRAAAAV128F421A322: {1: 6, 2: 4, 3: 2, 4: 2, 5: 5, 6: 3, 7: 1, 8: 1, 11: 1, 12: 2, 13: 3, 14: 1, 15: 1, 18: 2, 19: 2, 20: 2, 21: 2, 23: 4, 25: 1, 26: 2, 28: 1, 30: 1, 36: 2, 42: 1, 45: 1, 54: 2, 56: 1, 57: 1, 68: 1, 99: 1, 192: 2, 249: 1, 264: 1, 356: 1, 389: 1, 561: 1, 639: 1, 656: 1, 687: 1, 761: 1, 773: 1, 804: 1, 869: 2, 914: 1, 1035: 1, 1156: 1, 1221: 1, 1287: 1, 1364: 1, 1407: 1, 1533: 2, 1857: 1, 2096: 1, 2117: 1, 2482: 2, 2548: 1, 2705: 1, 2723: 1, 2868: 2, 2992: 2, 3455: 1, 3717: 1, 3851: 1, 4322: 1, 4382: 1, 4613: 1, 4713: 1, 4906: 1}
One example of the word dictionary
	TRAAAAV128F421A322: {'i': 6, 'the': 4, 'you': 2, 'to': 2, 'and': 5, 'a': 3, 'me': 1, 'it': 1, 'my': 1, 'is': 2, 'of': 3, 'your': 1, 'that': 1, 'are': 2, 'we': 2, 'am': 2, 'will': 2, 'for': 4, 'be': 1, 'have': 2, 'so': 1, 'this': 1, 'like': 2, 'de': 1, 'up': 1, 'was': 2, 'if': 1, '

For word embeddings, we will use FastText Pretrained Models, which handle out-of-vocabulary (OOV) words using subword information.

We will use the english embeddings from fasttext, downloaded from [here](https://fasttext.cc/docs/en/crawl-vectors.html)

In [7]:
# LOAD MODEL
# Info: https://fasttext.cc/docs/en/crawl-vectors.html
# !pip install fasttext
# fasttext.util.download_model('en', if_exists='ignore')  # English
# !pip install PyStemmer

# Import word2vec model
import fasttext
import fasttext.util
ft = fasttext.load_model('../match-api/models/cc.en.300.bin')
print(ft.get_dimension())

300


In [ ]:
# SAVE A DICTIONARY WITH SIMILAR WORDS FOR ALL 5000 WORDS IN LYRICS
import json
import os
from tqdm import tqdm
from lyrics_dictionary_initialisation import create_lyrics_similarity_dict, save_lyrics_similarity_dict

# Create the dictionary (or load it if already created)
lyrics_similarity_dict = create_lyrics_similarity_dict(all_words, ft, save_path='../match-api/data/lyrics_similarity_dict.json', batch_size=100)

# Save the lyrics_similarity_dict as a pickle for faster loading
save_lyrics_similarity_dict(lyrics_similarity_dict)